# 7. Calculating the Correlations between BTC and Other Coins

In [ ]:
from binance.client import Client
client = Client(
    'your_api_key', 
    'your_api_secret', 
    requests_params={'timeout': 30} # <-- This is the fix!
)

In [2]:
import pandas as pd
import requests
import time
import numpy as np

def fetch_crypto_data(symbol, interval, days=2):
    url = "https://api.binance.com/api/v3/klines"
    start_time = int(time.time() * 1000) - (days * 24 * 60 * 60 * 1000)
    params = {
        'symbol': symbol,
        'interval': interval,
        'startTime': start_time,
        'limit': 1000
    }
    response = requests.get(url, params=params)
    data = response.json()
    df = pd.DataFrame(data, columns=[
        'timestamp', 'open', 'high', 'low', 'close', 'volume', 
        'close_time', 'qav', 'num_trades', 'taker_base', 'taker_quote', 'ignore'
    ])
    df['close'] = df['close'].astype(float)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    return df[['timestamp', 'close']]

def calculate_correlation_and_sensitivity(base_df, target_df, window):
    merged = pd.merge(base_df, target_df, on='timestamp', suffixes=('_base', '_target'))
    merged['return_base'] = merged['close_base'].pct_change()
    merged['return_target'] = merged['close_target'].pct_change()
    merged = merged.dropna()
    correlation = merged['return_base'].corr(merged['return_target'])
    sensitivity = merged['return_target'].std() / merged['return_base'].std()
    trend_direction_score = 1 if merged['return_target'].tail(window).mean() > 0 else -1
    combined_score = correlation * sensitivity * trend_direction_score
    return correlation, sensitivity, trend_direction_score, combined_score

def suggest_trades(btc_df, target_df, metric, trend):
    target_df['Signal'] = 'Hold'
    target_df.loc[(target_df['close'] > target_df[metric]) & (trend == 'Positive'), 'Signal'] = 'Buy'
    target_df.loc[(target_df['close'] < target_df[metric]), 'Signal'] = 'Sell'
    return target_df

coins = ['USDTTRY', 'USDTARS', 'USDTCOP', 'BTCUSDT', 'ETHUSDT', 'DOGEUSDT',
       'USDCUSDT', 'SUIUSDT', 'SOLUSDT', 'XRPUSDT', 'FDUSDUSDT',
       'BIOUSDT', 'PEPEUSDT', 'ADAUSDT', 'ENAUSDT', 'TRXUSDT', 'XLMUSDT',
       'FDUSDTRY', 'BNBUSDT', 'PENGUUSDT', 'HBARUSDT', 'USDTBRL',
       'USUALUSDT', 'PHAUSDT', 'PNUTUSDT', 'LINKUSDT', 'NEIROUSDT',
       'ARBUSDT', 'SHIBUSDT', 'AVAXUSDT', 'WLDUSDT', 'BONKUSDT',
       'MOVEUSDT', 'AAVEUSDT', 'RSRUSDT', 'WIFUSDT', 'SUSHIUSDT',
       'UNIUSDT', 'LTCUSDT', 'STGUSDT', 'TAOUSDT', 'FLOKIUSDT', 'FTMUSDT',
       'GALAUSDT', 'TROYUSDT', 'CRVUSDT', 'ACTUSDT', 'DOTUSDT',
       'SANDUSDT', 'ZENUSDT']

results = []
btc_df = fetch_crypto_data('BTCUSDT', '15m', days=2)

for coin in coins:
    try:
        target_df = fetch_crypto_data(coin, '15m', days=2)
        correlation, sensitivity, trend_direction_score, combined_score = calculate_correlation_and_sensitivity(btc_df, target_df, 4)
        results.append({'Coin': coin, 'Correlation': correlation, 'Sensitivity': sensitivity, 'trend_direction_score':trend_direction_score , 'Combined Score': combined_score})
    except:
        continue

results_df = pd.DataFrame(results)

btc_trend = 'Positive' if btc_df['close'].iloc[-1] > btc_df['close'].iloc[-4] else 'Negative'
best_metric = 'EMA_20'

if btc_trend == 'Positive':
    best_coin = results_df.sort_values(['Combined Score'], ascending=False).iloc[0]['Coin']
else:
    best_coin = results_df.sort_values(
        ['trend_direction_score', 'Correlation', 'Sensitivity'],
        ascending=[True, False, False]
    ).iloc[0]['Coin']
    
best_coin_df = fetch_crypto_data(best_coin, '15m')
best_coin_df[best_metric] = best_coin_df['close'].ewm(span=int(best_metric.split('_')[1]), adjust=False).mean()
best_coin_df = suggest_trades(btc_df, best_coin_df, best_metric, btc_trend)

print(best_coin_df.tail())

              timestamp   close    EMA_20 Signal
187 2026-01-16 19:15:00  0.0262  0.026517   Sell
188 2026-01-16 19:30:00  0.0262  0.026487   Sell
189 2026-01-16 19:45:00  0.0262  0.026459   Sell
190 2026-01-16 20:00:00  0.0264  0.026454   Sell
191 2026-01-16 20:15:00  0.0264  0.026449   Sell


In [3]:
btc_trend

'Positive'